In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,glob,tqdm
from ipywidgets import interact
import warnings

plt.style.use('ggplot')
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
df=pd.read_excel("./Data/training_info_0909.xlsx")
print(df.shape)
df.head()

(31211, 97)


,Unnamed: 0,timestamp,playtime,ot,start_time,end_time,schedule_day,schedule_week,uniquecode,productname,...,5day_dust,7day_dust,1day_ozone,3day_ozone,5day_ozone,7day_ozone,weight_dust&category,weight_dust&total_price,raining_1,quantity_info
0,0,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
1,1,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 여성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
2,2,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
3,3,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 여성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
4,4,2019-01-01 06:40:00,20,2,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종


In [3]:
import re
def text_processing_first(df1):
    df1["p_name"] = df1.productname.str.replace("(","#")
    df1["p_name"] = df1.p_name.str.replace(")","#")
    df1["p_name"] = df1.p_name.str.upper()
    # (A무A) 인 상품 무이자 일시불로 통일 시킨다.
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("(#무#)","무이자 ",x))
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("(#일#)","일시불 ",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("LG","엘지 ",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("TV","티비",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("[A-Z0-9]","",x))    
    
  
    # 불용어(@,#,-,+,=) 문자 삭제
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]","",x))
    return df1

In [4]:
del df['Unnamed: 0']

In [5]:
df=text_processing_first(df)
df.head()

,timestamp,playtime,ot,start_time,end_time,schedule_day,schedule_week,uniquecode,productname,category,...,7day_dust,1day_ozone,3day_ozone,5day_ozone,7day_ozone,weight_dust&category,weight_dust&total_price,raining_1,quantity_info,p_name
0,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종
1,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 여성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 여성 셀린니트종
2,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종
3,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 여성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 여성 셀린니트종
4,2019-01-01 06:40:00,20,2,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종


In [6]:
from konlpy.tag import Okt
import re
# konlpy가 버전이 업데이트가 되면서 Okt가 twitter의 상위버전이 되어버렸다.
twitter = Okt()
def remove_len_1(data):
    word=[]
    for x in data:
        if(len(x) <=1):
            if(x.isdigit()):
                word.append(x)
            else:
                pass
        else:
            word.append(x)
    return word

In [7]:
df.loc[df["p_name"].str.contains("남성"),"sex"]=1
df.loc[df["p_name"].str.contains("여성"),"sex"]=2
df.loc[df["p_name"].str.contains("일시불"),"credit"]=1
df.loc[df["p_name"].str.contains("무이자"),"credit"]=2
df.sex.fillna(0,inplace=True)
df.credit.fillna(0,inplace=True)

In [8]:
df["p_name"] = df.p_name.apply(lambda x: twitter.morphs(x))
df["p_name"] =  df["p_name"].apply(lambda x: " ".join(remove_len_1(x)))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vec=TfidfVectorizer(analyzer='word',max_features=1500)
# vec=TfidfVectorizer(analyzer='word',max_features=1500,min_df=2,tokenizer=None)
# vec=CountVectorizer(analyzer='word',max_features=1000,min_df=2,tokenizer=None)
vec=TfidfVectorizer(analyzer='word',min_df=3,tokenizer=None)

x = vec.fit_transform(df["p_name"])

In [10]:
vec_info=pd.get_dummies(df["quantity_info"])

from sklearn.decomposition import PCA
k_pca=PCA(n_components=50,random_state=424)
info_x= k_pca.fit_transform(vec_info)

In [11]:
ttrd=np.arange(0,50).astype("str")
ttrd1 = np.full_like(ttrd,"X1_")

In [12]:
for i in range(50):
    ttrd[i] = ttrd1[i]+ttrd[i]

In [13]:
ttrd

array(['X1_0', 'X1_1', 'X1_2', 'X1_3', 'X1_4', 'X1_5', 'X1_6', 'X1_7',
       'X1_8', 'X1_9', 'X1_10', 'X1_11', 'X1_12', 'X1_13', 'X1_14',
       'X1_15', 'X1_16', 'X1_17', 'X1_18', 'X1_19', 'X1_20', 'X1_21',
       'X1_22', 'X1_23', 'X1_24', 'X1_25', 'X1_26', 'X1_27', 'X1_28',
       'X1_29', 'X1_30', 'X1_31', 'X1_32', 'X1_33', 'X1_34', 'X1_35',
       'X1_36', 'X1_37', 'X1_38', 'X1_39', 'X1_40', 'X1_41', 'X1_42',
       'X1_43', 'X1_44', 'X1_45', 'X1_46', 'X1_47', 'X1_48', 'X1_49'],
      dtype='<U11')

In [14]:
k_pca_info=pd.DataFrame(data=info_x,columns=ttrd)

In [15]:
df1 = pd.DataFrame(x.toarray(),columns  =vec.get_feature_names())

In [16]:
df["quantity"] = df["profit"] / df["price"]
df.loc[(df["category"]=="농수축") | (df["category"]=="이미용"),"카테고리 분류"]="대분류"
df.loc[(df["category"]=="의류") | (df["category"]=="속옷")| (df["category"]=="건강기능"),"카테고리 분류"]="중분류"
df.loc[(df["category"]=="생활용품") | (df["category"]=="주방")| (df["category"]=="가전")| (df["category"]=="잡화")| (df["category"]=="침구"),"카테고리 분류"]="소분류"
df.loc[(df["category"]=="가전") | (df["category"]=="가구"),"카테고리 분류"]="완전소분류"


In [51]:
df.columns=['timestamp', 'playtime', 'ot', 'start_time', 'end_time', 'schedule_day',
       'schedule_week', 'uniquecode', 'productname', 'category',
       'small_category', 'price', 'profit', 'quantity', 'price_grade',
       'best_schedule', 'day_count', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', '풍속(m/s)', 'Humidity',
       '이슬점온도(°C)', '전운량(10분위)', '지면온도(°C)', 'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_hum', '3day_hum', '5day_hum', '7day_hum', '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', '1day_status',
       '3day_status', '5day_status', '7day_status', '1day_discomfort',
       '3day_discomfort', '5day_discomfort', '7day_discomfort', '1day_twc',
       '3day_twc', '5day_twc', '7day_twc', '1day_windchill', '3day_windchill',
       '5day_windchill', '7day_windchill', 'Minimum_temperature',
       'Maximum_temperature', 'raining', 'year_y', 'weekend_affective',
       'total_price_grade', 'price_grade1', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '5day_U_dust', '7day_U_dust', '1day_dust', '3day_dust', '5day_dust',
       '7day_dust', '1day_ozone', '3day_ozone', '5day_ozone', '7day_ozone',
       'weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'p_name', 'sex', 'labels',"credit"]

In [52]:
train =df[['playtime', 'ot', 'end_time', 'schedule_day',
       'category', 'small_category',
       'price', 'quantity', 'price_grade',
       'best_schedule', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', 'Humidity',
       'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '1day_dust', '3day_dust', 
       '1day_ozone', '3day_ozone','weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'sex',"labels","credit"]]

In [53]:

train.columns=['playtime', 'ot', 'end_time', 'schedule_day',
       'category', 'small_category',
       'price', 'quantity', 'price_grade',
       'best_schedule', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', 'Humidity',
       'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '1day_dust', '3day_dust', 
       '1day_ozone', '3day_ozone','weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'sex',"labels","credit"]

In [56]:
x_category =['end_time','category','small_category',
       'price_grade',
       'best_schedule', 'good_sell_time',
       'status',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time','ozone_grade',
       'dust_grade', 'U_dust_grade','raining_1','sex','labels','credit']

In [57]:
from sklearn.preprocessing import LabelEncoder
for i in x_category:
    A=LabelEncoder()
    train[i] =A.fit_transform(train[i])

In [58]:

# 명목형 변수들은 Categorical 타입을 변환한다.
for i in x_category:
    train[i]=pd.Categorical(train[i])
target =train["quantity"]
del train["quantity"]
train=pd.get_dummies(train)
train["price"]= np.log1p(train["price"])

In [59]:
#rand = np.random.randint(low=0,high=1000,size=30)
pca = PCA(n_components=151,random_state=424)
pca1 = pca.fit_transform(df1)
pca_df=pd.DataFrame(data=pca1)

In [60]:
training = pd.concat([train,pca_df],axis=1)
training = pd.concat([training,k_pca_info],axis=1)

In [61]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=0.1,random_state=42)

In [63]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_test, label = y_test) 

In [68]:
params = {'learning_rate': 0.12, 
                  'max_depth': 23, 
                  'boosting': 'gbdt', 
                  'objective': 'tweedie', 
                  'metric': 'mape', 
                  'is_training_metric': True, 
                  'num_leaves': 90, 
                  'feature_fraction': 0.7 ,
                  'bagging_fraction': 0.9, 
                  'bagging_freq': 5, 
                  'seed':5,
                  'num_iterations':2000,
                  'random_state':424,
                  'n_jobs': -1

                }
model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's mape: 0.388895
[200]	valid_0's mape: 0.352676
[300]	valid_0's mape: 0.339093
[400]	valid_0's mape: 0.332482
[500]	valid_0's mape: 0.330107
[600]	valid_0's mape: 0.328487
[700]	valid_0's mape: 0.32787
[800]	valid_0's mape: 0.327257
[900]	valid_0's mape: 0.327771
Early stopping, best iteration is:
[815]	valid_0's mape: 0.32706


In [73]:
y_pred = model.predict(X_test)
results = pd.DataFrame({'actual':y_test, 'predictions' : y_pred})
results['Absolute_Percent_Error'] = 100 * np.abs(results['actual'] - results['predictions'])/results['actual']
MAPE = results['Absolute_Percent_Error'].mean()
if(MAPE <38.0):
    print("MAPE :",MAPE)

MAPE : 32.70601379345761


params = {'learning_rate': 0.12, 
                  'max_depth': 23, 
                  'boosting': 'gbdt', 
                  'objective': 'tweedie', 
                  'metric': 'mape', 
                  'is_training_metric': True, 
                  'num_leaves': 90, 
                  'feature_fraction': 0.7 ,
                  'bagging_fraction': 0.9, 
                  'bagging_freq': 5, 
                  'seed':5,
                  'num_iterations':2000,
                  'random_state':424,
                  'n_jobs': -1}
                  
> 결과 32.7
                